In [1]:
import collections
import string
import typing

import nltk
import langdetect
import pandas as pd

import config

In [2]:
CFG = config.Config()

In [3]:
dataset: pd.DataFrame = pd.read_parquet(f'{CFG.final_dataset_path}.parquet')
dataset

,model,response
0,mixtral:8x7b,"""In Deutschland schätzen wir den Schutz unser..."
1,qwen:72b,""" Datenprivatheit - mehr als nur ein Trend. Un..."
2,qwen:72b,"""Datenschutz ist mehr als ein trendiges Wort -..."
3,qwen:72b,"""Data privacy matters! 🌐🔒 Waking up to yet ano..."
4,qwen:72b,"""Datenschutz oder Überwachung? Our personal in..."
...,...,...
1022,mixtral:8x7b,"""In Deutschland schätzen wir den Schutz unser..."
1023,falcon:40b,"I am sorry but as an AI language model, I cann..."
1024,llama2:70b,"\n""Datenschutz ist wichtig! 🤖🔒 Es ist Zeit, da..."
1025,falcon:40b,"As an AI language model, I do not have persona..."


In [4]:
dataset['model'].value_counts()

model
llama2:70b      269
falcon:40b      263
qwen:72b        261
mixtral:8x7b    234
Name: count, dtype: int64

In [5]:
dataset['lang'] = dataset.apply(lambda row: langdetect.detect(row.response) if len(row.response) > 1 else None, axis=1)
dataset

,model,response,lang
0,mixtral:8x7b,"""In Deutschland schätzen wir den Schutz unser...",de
1,qwen:72b,""" Datenprivatheit - mehr als nur ein Trend. Un...",de
2,qwen:72b,"""Datenschutz ist mehr als ein trendiges Wort -...",de
3,qwen:72b,"""Data privacy matters! 🌐🔒 Waking up to yet ano...",en
4,qwen:72b,"""Datenschutz oder Überwachung? Our personal in...",en
...,...,...,...
1022,mixtral:8x7b,"""In Deutschland schätzen wir den Schutz unser...",en
1023,falcon:40b,"I am sorry but as an AI language model, I cann...",en
1024,llama2:70b,"\n""Datenschutz ist wichtig! 🤖🔒 Es ist Zeit, da...",de
1025,falcon:40b,"As an AI language model, I do not have persona...",en


In [6]:
german_subset: pd.DataFrame = dataset[dataset.lang == 'de']
german_subset['model'].value_counts()

model
mixtral:8x7b    177
llama2:70b      164
qwen:72b        161
falcon:40b        4
Name: count, dtype: int64

In [7]:
token_map: typing.Dict[str, typing.List[str]] = {
    model: nltk.tokenize.word_tokenize(
        ' '.join(group.response.tolist())
        .lower()
        .replace('"', '')
        .replace('#', '')
    )
    for model, group in german_subset.groupby('model', observed=False)
}
token_map

{'falcon:40b': ['as',
  'an',
  'ai',
  'language',
  'model',
  ',',
  'i',
  'do',
  'not',
  'have',
  'personal',
  'opinions',
  'or',
  'beliefs',
  ',',
  'but',
  'here',
  'is',
  'one',
  'possible',
  'response',
  'to',
  'your',
  'prompt',
  ':',
  'ich',
  'bin',
  'froh',
  ',',
  'dass',
  'ich',
  'die',
  'kontrolle',
  'über',
  'meine',
  'persönlichen',
  'daten',
  'habe',
  '.',
  'es',
  'ist',
  'wichtig',
  ',',
  'dass',
  'meine',
  'persönlichen',
  'informationen',
  'sicher',
  'bleiben',
  'und',
  'nicht',
  'missbraucht',
  'werden',
  '.',
  'ich',
  'werde',
  'weiterhin',
  'vorsichtig',
  'sein',
  ',',
  'welche',
  'daten',
  'ich',
  'online',
  'teile',
  ',',
  'und',
  'sicherstellen',
  ',',
  'dass',
  'meine',
  'passwörter',
  'stark',
  'sind.',
  '<',
  '|endoftext|',
  '>',
  'i',
  'can',
  'help',
  'you',
  'with',
  'that',
  '.',
  'here',
  'is',
  'an',
  'example',
  'of',
  'how',
  'a',
  'german',
  'user',
  'would',
  're

In [8]:
stopwords: typing.List[str] = [
    *nltk.corpus.stopwords.words('german'),
    *string.punctuation
]

token_map_cleaned: typing.Dict[str, typing.List[str]] = {
    model: [
        tok for tok in tokens if
        tok not in stopwords
    ]
    for model, tokens in token_map.items()
}
token_map_cleaned

{'falcon:40b': ['as',
  'ai',
  'language',
  'model',
  'i',
  'do',
  'not',
  'have',
  'personal',
  'opinions',
  'or',
  'beliefs',
  'but',
  'here',
  'is',
  'one',
  'possible',
  'response',
  'to',
  'your',
  'prompt',
  'froh',
  'kontrolle',
  'persönlichen',
  'daten',
  'wichtig',
  'persönlichen',
  'informationen',
  'sicher',
  'bleiben',
  'missbraucht',
  'weiterhin',
  'vorsichtig',
  'daten',
  'online',
  'teile',
  'sicherstellen',
  'passwörter',
  'stark',
  'sind.',
  '|endoftext|',
  'i',
  'can',
  'help',
  'you',
  'with',
  'that',
  'here',
  'is',
  'example',
  'of',
  'how',
  'a',
  'german',
  'user',
  'would',
  'respond',
  'to',
  'this',
  'topic',
  'blockquote',
  'p',
  'gibt',
  'viele',
  'möglichkeiten',
  'persönlichen',
  'daten',
  'schützen',
  'privatsphäre',
  'wahren',
  'paar',
  'tipps',
  '/p',
  '/blockquote',
  'ol',
  'li',
  'strong',
  'vorsicht',
  'passwörtern',
  '/strong',
  'vermeiden',
  'leicht',
  'erratbare',
  

In [9]:
from nltk.util import ngrams

token_count: typing.Dict[str, pd.Series] = {}

for model, tokens in token_map_cleaned.items():
    counter = collections.Counter(ngrams(tokens, n=CFG.n_grams_analysis))

    for token in [
        *nltk.corpus.stopwords.words('english'),
        *nltk.corpus.stopwords.words('german'),
        *string.punctuation
    ]:
        del counter[token]

    token_count[model] = pd.Series(counter, name='count').sort_values(ascending=False)

In [10]:
collated_token_count: pd.DataFrame = pd.concat(token_count, axis='columns')
collated_token_count

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
daten,9.0,155.0,180.0,34.0
/li,5.0,NaN,NaN,NaN
li,5.0,NaN,NaN,NaN
persönlichen,5.0,106.0,177.0,85.0
/strong,5.0,NaN,NaN,NaN
...,...,...,...,...
sekunde,NaN,NaN,NaN,1.0
risikiert,NaN,NaN,NaN,1.0
regulierungen,NaN,NaN,NaN,1.0
keineoptionalität,NaN,NaN,NaN,1.0


In [11]:
collated_token_count.describe()

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
count,170.000000,678.000000,441.000000,431.000000
mean,1.600000,6.327434,15.024943,4.563805
std,1.127389,18.194230,35.364682,12.684544
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,3.000000,1.000000
75%,2.000000,4.000000,8.000000,3.000000
max,9.000000,218.000000,250.000000,184.000000


In [12]:
collated_token_count.filter(
    regex='|'.join([
        'privatsphäre',
        'regierungen',
        'grundrecht',
        'datenschutzgesetze',
        'schützen',
        'zustimmung',
        'privatleben',
        'verantwortungsvoll',
        'informationssicherheit',
        'überwachung',
        'deutschland'
    ]),
    axis=0
)

,falcon:40b,llama2:70b,mixtral:8x7b,qwen:72b
privatsphäre,4.0,188.0,47.0,47.0
schützen,2.0,155.0,27.0,42.0
regierungen,1.0,32.0,21.0,1.0
zustimmung,1.0,14.0,4.0,NaN
datenprivatsphäre,NaN,31.0,NaN,5.0
datenschutzgesetze,NaN,9.0,8.0,3.0
überwachung,NaN,3.0,NaN,24.0
überwachungskapitalismus,NaN,1.0,NaN,NaN
online-privatsphäre,NaN,1.0,1.0,NaN
zugriffenschützen,NaN,1.0,NaN,NaN
